# 投票分类器

In [1]:
# 假设你已经训练好了一些分类器，每个分类器的准确率约为80%。
# 大概包括一个逻辑回归分类器、一个SVM分类器、一个随机森林分类器、一个K-近邻分类器，或许还有更多
# 这时，要创建出一个更好的分类器，最简单的办法就是聚合每个分类器的预测，
# 然后将得票最多的结果作为预测类别。这种大多数投票分类器被称为硬投票分类器

In [2]:
# 当预测器尽可能互相独立时，集成方法的效果最优。
# 获得多种分类器的方法之一就是使用不同的算法进行训练。
# 这会增加它们犯不同类型错误的机会，从而提升集成的准确率

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split


X, y = make_moons(n_samples=1000, noise=0.20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(estimators=[('lr', log_clf),
                                          ('rf', rnd_clf),
                                          ('svc', svm_clf)],
                              voting='hard')

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [3]:
# 看一下测试集上每个分类器的精度
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.865
RandomForestClassifier 0.965
SVC 0.96
VotingClassifier 0.955


In [4]:
# 如果所有分类器都能够估算出类别的概率（即有predict_proba（）方法），
# 那么你可以将概率在所有单个分类器上平均，然后让Scikit-Learn给出平均概率最高的类别作为预测。
# 这被称为软投票法。通常来说，它比硬投票法的表现更优，
# 因为它给予那些高度自信的投票更高的权重。
# 而所有你需要做的就是用voting="soft"代替voting="hard"，
# 并确保所有分类器都可以估算出概率。

# bagging和pasting

In [5]:
# 前面提到，获得不同种类分类器的方法之一是使用不同的训练算法。
# 还有另一种方法是每个预测器使用的算法相同，但是在不同的训练集随机子集上进行训练。
# 采样时如果将样本放回，这种方法叫作bagging[1]（bootstrap aggregating[2]的缩写，也叫自举汇聚法）。
# 采样时样本不放回，这种方法则叫作pasting

In [6]:
# Scikit-Learn提供了一个简单的API，可用BaggingClassifier类进行bagging和pasting（或BaggingRegressor用于回归）

# 如果基本分类器可以估计类别概率（如果它具有predict_proba（）方法），
# 则BaggingClassifier自动执行软投票而不是硬投票，在决策树分类器中就是这种情况

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), 
                            n_estimators=500,           # 500个决策树分类器的集成
                            max_samples=100,            # 每次从训练集中随机采样100个训练实例进行训练
                            bootstrap=True,             # 放回抽样，不放回抽样为False
                            n_jobs=-1)                  # 使用所有cpu内核进行训练

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

                            包外评估                            

In [7]:
# 对于任意给定的预测器，使用bagging，有些实例可能会被采样多次，
# 而有些实例则可能根本不被采样。BaggingClassifier默认采样m个训练实例，
# 然后放回样本（bootstrap=True），m是训练集的大小。
# 这意味着对每个预测器来说，平均只对63%的训练实例进行采样。
# 剩余37%未被采样的训练实例称为包外（oob）实例

# 由于预测器在训练过程中从未看到oob实例，因此可以在这些实例上进行评估，而无须单独的验证集。
# 你可以通过平均每个预测器的oob评估来评估整体

In [8]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(),
                            n_estimators=500,
                            bootstrap=True,
                            n_jobs=-1,
                            oob_score=True)         # 请求在训练结束后自动进行包外评估

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.955

In [9]:
# 根据此oob评估，此BaggingClassifier能在测试集上达到约95%的准确率。让我们验证一下：

from sklearn.metrics import accuracy_score

y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.975

# 随机补丁和随机子空间

In [10]:
# BaggingClassifier类也支持对特征进行采样。采样由两个超参数控制：max_features和bootstrap_features。
# 它们的工作方式与max_samples和bootstrap相同，但用于特征采样而不是实例采样。
# 因此，每个预测器将用输入特征的随机子集进行训练。
# 这对于处理高维输入（例如图像）特别有用。
# 对训练实例和特征都进行抽样，这称为随机补丁方法。
# 而保留所有训练实例（即bootstrap=False并且max_samples=1.0）
# 但是对特征进行抽样（即bootstrap_features=True并且/或max_features<1.0），这被称为随机子空间法。
# 对特征抽样给预测器带来更大的多样性，所以以略高一点的偏差换取了更低的方差

# 随机森林

In [11]:
# 前面已经提到，随机森林是决策树的集成，通常用bagging（有时也可能是pasting）方法训练，训练集大小通过max_samples来设置。
# 除了先构建一个BaggingClassifier然后将其传输到DecisionTreeClassifier，
# 还有一种方法就是使用RandomForestClassifier类，这种方法更方便，对决策树更优化

# RandomForestClassifier具有DecisionTreeClassifier的所有超参数（以控制树的生长方式），
# 以及BaggingClassifier的所有超参数来控制集成本身

from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf.predict(X_test)

                    极端随机树                  

In [ ]:
# 如前所述，在随机森林里单棵树的生长过程中，每个节点在分裂时仅考虑到了一个随机子集所包含的特征。
# 如果我们对每个特征使用随机阈值，而不是搜索得出的最佳阈值（如常规决策树），
# 则可能让决策树生长得更加随机

# 使用Scikit-Learn的ExtraTreesClassifier类可以创建一个极端随机树分类器。
# 它的API与RandomForestClassifier类相同

# 通常来说，很难预先知道一个RandomForestClassifier类是否会比一个ExtraTreesClassifier类更好或是更差。
# 唯一的方法是两种都尝试一遍，然后使用交叉验证（还需要使用网格搜索调整超参数）进行比较

                    特征重要性                  

$$p_{i,k}是某类别所占比例$$
$$G_i=1-\sum_{k=1}^np_{i,k}^2$$

In [ ]:
# 通过计算节点分裂前后基尼系数的改变量来衡量该特征的重要性

# 随机森林的另一个好特性是它们使测量每个特征的相对重要性变得容易。
# Scikit-Learn通过查看使用该特征的树节点平均（在森林中的所有树上）减少不纯度的程度来衡量该特征的重要性。
# 更准确地说，它是一个加权平均值，其中每个节点的权重等于与其关联的训练样本的数量

In [12]:
# Scikit-Learn会在训练后为每个特征自动计算该分数，然后对结果进行缩放以使所有重要性的总和等于1。
# 你可以使用feature_importances_变量来访问结果

from sklearn.datasets import load_iris

iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])

for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.09912150353254087
sepal width (cm) 0.023732187293302842
petal length (cm) 0.44020494187074427
petal width (cm) 0.4369413673034121


# 提升法

                    AdaBoost                    

In [ ]:
# 当训练AdaBoost分类器时，该算法首先训练一个基础分类器（例如决策树），并使用它对训练集进行预测。
# 然后，该算法会增加分类错误的训练实例的相对权重。
# 然后，它使用更新后的权重训练第二个分类器，并再次对训练集进行预测，更新实例权重，以此类推

# 这种依序学习技术有一个重要的缺陷就是无法并行

$$每个实例权重w^{(i)}最初设置为\frac {1}{m}$$
第j个预测器的加权误差率：
$$\begin{aligned}
r_j=\frac{\sum_{i=1,\ \hat{y}_j^{(i)}\neq y^{(i)}}^mw^{(i)}}{\sum_{i=1}^mw^{(i)}}
\end{aligned}$$
预测器权重：
$$\alpha_j=\eta\log\frac{1-r_j}{r_j}$$
权重更新规则：
$$\begin{aligned}\text{对于 }i&=1,2,\cdots,m\\w^{(i)}&\leftarrow\begin{cases}w^{(i)},&\text{如果}\hat{y}_j^{(i)}=y^{(i)}\\w^{(i)}\exp(\alpha_j),&\text{如果}\hat{y}_j^{(i)}\neq y^{(i)}&\end{cases}\end{aligned}$$

In [ ]:
# 预测器的权重αj通过公式2来计算，其中η是学习率超参数（默认为1）。
# 预测器的准确率越高，其权重就越高。如果它只是随机猜测，则其权重接近于零。
# 但是，如果大部分情况下它都是错的（也就是准确率比随机猜测还低），那么它的权重为负

# 预测的时候，AdaBoost就是简单地计算所有预测器的预测结果，并使用预测器权重αj对它们进行加权。
# 最后，得到大多数加权投票的类就是预测器给出的预测类

In [ ]:
# Scikit-Learn使用的其实是AdaBoost的一个多分类版本，叫作SAMME（基于多类指数损失函数的逐步添加模型）。
# 当只有两类时，SAMME即等同于AdaBoost。此外，如果预测器可以估算类概率（即具有predict_proba（）方法），
# Scikit-Learn会使用一种SAMME的变体，称为SAMME.R（R代表“Real”），它依赖的是类概率而不是类预测，通常表现更好。

from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                              n_estimators=200,
                              algorithm="SAMME.R",
                              learning_rate=0.5)

ada_clf.fit(X_train, y_train)

# 如果你的AdaBoost集成过度拟合训练集，你可以试试减少估算器数量，或是提高基础估算器的正则化程度

                    梯度提升                    

In [ ]:
# 另一个非常受欢迎的提升法是梯度提升。与AdaBoost一样，梯度提升也是逐步在集成中添加预测器，
# 每一个都对其前序做出改正。不同之处在于，它不是像AdaBoost那样在每个迭代中调整实例权重，
# 而是让新的预测器针对前一个预测器的残差进行拟合

In [13]:
# # 我们来看一个简单的回归示例，使用决策树作为基础预测器（梯度提升当然也适用于回归任务），
# # 这被称为梯度树提升或者是梯度提升回归树（GBRT）。
# # 首先，在训练集（比如带噪声的二次训练集）上拟合一个DecisionTreeRegressor：


# import numpy as np
# from sklearn.tree import DecisionTreeRegressor

# X = np.arange(-100,100,0.1)
# y = X**2 + np.random.rand(2000,)
# X = X.reshape(2000, 1)
# y = y.reshape(2000, 1)

# tree_reg1 = DecisionTreeRegressor(max_depth=2)
# tree_reg1.fit(X, y)

# y2 = y - tree_reg1.predict(X)
# tree_reg2 = DecisionTreeRegressor(max_depth=2)
# tree_reg2.fit(X, y2)

# y3 = y2 - tree_reg2.predict(X)
# tree_reg3 = DecisionTreeRegressor(max_depth=2)
# tree_reg3.fit(X, y3)

# y_pred = sum(tree.predict(X) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [20]:
# 训练GBRT集成有个简单的方法，就是使用Scikit-Learn的GradientBoostingRegressor类。
# 与RandomForestRegressor类似，它具有控制决策树生长的超参数（例如max_depth、min_samples_leaf等），
# 以及控制集成训练的超参数，例如树的数量（n_estimators）。
# 以下代码可创建上面的集成

from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X, y)

# 超参数learning_rate对每棵树的贡献进行缩放。如果你将其设置为低值，比如0.1，则需要更多的树来拟合训练集，
# 但是预测的泛化效果通常更好，这是一种被称为收缩的正则化技术

C:\Users\AKSJF\AppData\Roaming\Python\Python311\site-packages\sklearn\ensemble\_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [ ]:
# 实际上，要实现提前停止法，不一定需要先训练大量的树，然后再回头找最优的数字，
# 还可以提前停止训练。设置warm_start=True，
# 当fit（）方法被调用时，Scikit-Learn会保留现有的树，从而允许增量训练。
# 以下代码会在验证误差连续5次迭代未改善时，直接停止训练：

from sklearn.metrics import mean_squared_error


gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break # early stopping

In [ ]:
# 值得注意的是，流行的Python库XGBoost（该库代表Extreme
# Gradient Boosting）中提供了梯度提升的优化实现，该软件包最初是
# 由Tianqi Chen作为分布式（深度）机器学习社区（DMLC）的一部分开
# 发的，其开发目标是极快、可扩展和可移植。实际上，XGBoost通常是
# ML竞赛中获胜的重要组成部分。XGBoost的API与Scikit-Learn的非常相
# 似：

import xgboost
xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

# XGBoost还提供了一些不错的特性，例如自动处理提前停止：
xgb_reg.fit(X_train, y_train,
eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)

                    堆叠法                  